# Dreaming of Mars
- Creating DeepDreams from Mars landscape images from Curiosity Rover
- Code adapted from [Tensorflow DeepDream Tutorial](https://www.tensorflow.org/tutorials/generative/deepdream)

<img src="https://media1.tenor.com/images/ac6dbff8fb98d0cd71930685e04b3e7e/tenor.gif" alt="Mars Deep Dream Demo" width="500">

In [23]:
import numpy as np

import PIL.Image

from random import randint
from time import strftime, gmtime

import IPython.display as display

import cv2 as cv

import tensorflow as tf
from tensorflow.keras.preprocessing import image

# Get Image From NASA Mars Images API

In [24]:
url = 'https://mars.nasa.gov/msl-raw-images/msss/01590/mcam/1590MR0081020030800438E01_DXXX.jpg'

In [25]:
# Download an image and read it into a NumPy array.
def download(url, max_dim=None):
  name = url.split('/')[-1]
  image_path = tf.keras.utils.get_file(name, origin=url)
  img = PIL.Image.open(image_path)
  if max_dim:
    img.thumbnail((max_dim, max_dim))
  return np.array(img)

# Turn a normalized image array back into an image
def deprocess(img):
  img = 255*(img + 1.0)/2.0
  return tf.cast(img, tf.uint8)

def get_save_name(buffer="start"):
    name = buffer + strftime("%X", gmtime())
    name += str(randint(0, 1000)) + ".png"
    return name

# Display an image
def show(img, name):
    img_PIL = PIL.Image.fromarray(np.array(img))
#     img_PIL.save("1-3/" + get_save_name(name))
    display.display(img_PIL)


# Downsizing the image makes it easier to work with.
original_img = download(url, 400)

# Set-up Neural Network Model

In [26]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')
history = []

In [27]:
# Maximize the activations of these layers
names = ['mixed1', 'mixed3']
layers = [base_model.get_layer(name).output for name in names]

# Create the feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

In [28]:
def calc_loss(img, model):
  # Pass forward the image through the model to retrieve the activations.
  # Converts the image into a batch of size 1.
  img_batch = tf.expand_dims(img, axis=0)
  layer_activations = model(img_batch)
  if len(layer_activations) == 1:
    layer_activations = [layer_activations]

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)

  return  tf.reduce_sum(losses)

In [29]:
def random_roll(img, maxroll):
  # Randomly shift the image to avoid tiled boundaries.
  shift = tf.random.uniform(shape=[2], minval=-maxroll, maxval=maxroll, dtype=tf.int32)
  img_rolled = tf.roll(img, shift=shift, axis=[0,1])
  return shift, img_rolled

In [30]:
shift, img_rolled = random_roll(np.array(original_img), 512)

Here is a tiled equivalent of the `deepdream` function defined earlier:

In [31]:
class TiledGradients(tf.Module):
  def __init__(self, model):
    self.model = model

  @tf.function(
      input_signature=(
        tf.TensorSpec(shape=[None,None,3], dtype=tf.float32),
        tf.TensorSpec(shape=[], dtype=tf.int32),)
  )
  def __call__(self, img, tile_size=512):
    shift, img_rolled = random_roll(img, tile_size)

    # Initialize the image gradients to zero.
    gradients = tf.zeros_like(img_rolled)
    
    # Skip the last tile, unless there's only one tile.
    xs = tf.range(0, img_rolled.shape[0], tile_size)[:-1]
    if not tf.cast(len(xs), bool):
      xs = tf.constant([0])
    ys = tf.range(0, img_rolled.shape[1], tile_size)[:-1]
    if not tf.cast(len(ys), bool):
      ys = tf.constant([0])

    for x in xs:
      for y in ys:
        # Calculate the gradients for this tile.
        with tf.GradientTape() as tape:
          # This needs gradients relative to `img_rolled`.
          # `GradientTape` only watches `tf.Variable`s by default.
          tape.watch(img_rolled)

          # Extract a tile out of the image.
          img_tile = img_rolled[x:x+tile_size, y:y+tile_size]
          loss = calc_loss(img_tile, self.model)

        # Update the image gradients for this tile.
        gradients = gradients + tape.gradient(loss, img_rolled)

    # Undo the random shift applied to the image and its gradients.
    gradients = tf.roll(gradients, shift=-shift, axis=[0,1])

    # Normalize the gradients.
    gradients /= tf.math.reduce_std(gradients) + 1e-8 

    return gradients 

In [32]:
get_tiled_gradients = TiledGradients(dream_model)

In [33]:
def run_deep_dream_with_octaves(img, steps_per_octave=100, step_size=0.01, 
                                octaves=range(-2,3), octave_scale=1.3):
  base_shape = tf.shape(img)
  img = tf.keras.preprocessing.image.img_to_array(img)
  img = tf.keras.applications.inception_v3.preprocess_input(img)

  initial_shape = img.shape[:-1]
  img = tf.image.resize(img, initial_shape)
  for octave in octaves:
    # Scale the image based on the octave
    new_size = tf.cast(tf.convert_to_tensor(base_shape[:-1]), tf.float32)*(octave_scale**octave)
    img = tf.image.resize(img, tf.cast(new_size, tf.int32))

    for step in range(steps_per_octave):
      gradients = get_tiled_gradients(img)
      img = img + gradients*step_size
      img = tf.clip_by_value(img, -1, 1)

      if step % 10 == 0:
        display.clear_output(wait=True)
        history.append(deprocess(img))
        print(len(history))
    
  result = deprocess(img)
  return result

In [34]:
img = run_deep_dream_with_octaves(img=original_img, step_size=0.01)

display.clear_output(wait=True)

50


# Generate Video From Dream

In [50]:
def process_frame_for_video(frame, text="", size=(400, 400)):
    frame = cv.resize(np.array(frame), size)
    frame = cv.cvtColor( frame, cv.COLOR_RGB2BGR )
    if len(text) > 0:
        text_bottom_left = (int(size[0]/5), int(size[0]/2))
        cv.putText( frame, text, text_bottom_left, cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    return frame

In [53]:
fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter("mars_dream.mp4", fourcc, 3.0, (400, 400))

for i in range(3):
    out.write(process_frame_for_video(history[0], text="Sweet Dreams,",))
    
for i in range(4):
    out.write(process_frame_for_video(history[0], text="Curiosity Rover"))

for i in range(len(history)):
    out.write(process_frame_for_video(history[i]))
    
out.release()
cv.destroyAllWindows()